# Sales Prediction 

In [1]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('..', 'scripts')))

In [9]:

import data_processing as pd 
import eda as eda


In [3]:
# load data
train_data = eda.load_data('../data/train.csv')
store_data = eda.load_data('../data/store.csv')
sample_data = eda.load_data('../data/sample_submission.csv')
test_data = eda.load_data('../data/test.csv')


2025-01-14 21:32:24,200 - INFO - Loading data...
/home/abel/CODE/KiFiya/10-Academy-Week-4/scripts/eda.py:16: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(data_path)
2025-01-14 21:32:25,789 - INFO - Data loaded successfully from ../data/train.csv
2025-01-14 21:32:25,791 - INFO - Loading data...
2025-01-14 21:32:25,799 - INFO - Data loaded successfully from ../data/store.csv
2025-01-14 21:32:25,801 - INFO - Loading data...
2025-01-14 21:32:25,814 - INFO - Data loaded successfully from ../data/sample_submission.csv
2025-01-14 21:32:25,816 - INFO - Loading data...
2025-01-14 21:32:25,876 - INFO - Data loaded successfully from ../data/test.csv


In [5]:

data = pd.load_data('../data/train.csv')
data = pd.extract_features(data)
data = pd.handle_missing_values(data)

/home/abel/CODE/KiFiya/10-Academy-Week-4/scripts/data_processing.py:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)
/home/abel/CODE/KiFiya/10-Academy-Week-4/scripts/data_processing.py:37: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return data.fillna(method='ffill')


In [8]:

# Check for non-numeric values in each column
import pandas as pad
feature_columns = ['Weekday', 'IsWeekend', 'DaysToHoliday', 'DaysAfterHoliday',
                       'BeginningOfMonth', 'MidMonth', 'EndOfMonth', 'Season', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday']
for col in feature_columns:
    if not pad.api.types.is_numeric_dtype(data[col]):
        print(f"Non-numeric values found in column: {col}")
        print(data[col].unique())


Non-numeric values found in column: StateHoliday
['0' 'a' 'b' 'c' 0]


In [10]:

data = pd.convert_categorical(data)

/home/abel/CODE/KiFiya/10-Academy-Week-4/scripts/data_processing.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['StateHoliday'].fillna(0, inplace=True)


In [11]:

feature_columns = ['Weekday', 'IsWeekend', 'DaysToHoliday', 'DaysAfterHoliday',
                   'BeginningOfMonth', 'MidMonth', 'EndOfMonth', 'Season', 'Open', 'Promo', 'StateHoliday']
# Scale features
data = pd.scale_features(data, feature_columns)
data.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,Weekday,IsWeekend,DaysToHoliday,DaysAfterHoliday,BeginningOfMonth,MidMonth,EndOfMonth,Season,SchoolHoliday_1
0,1,5,2015-07-31,5263,555,0.452399,1.273237,-0.159217,0.501484,-0.630672,-1.734123,0.0,-0.700515,-0.700517,1.388213,0.599358,True
1,2,5,2015-07-31,6064,625,0.452399,1.273237,-0.159217,0.501484,-0.630672,-1.734123,0.0,-0.700515,-0.700517,1.388213,0.599358,True
2,3,5,2015-07-31,8314,821,0.452399,1.273237,-0.159217,0.501484,-0.630672,-1.734123,0.0,-0.700515,-0.700517,1.388213,0.599358,True
3,4,5,2015-07-31,13995,1498,0.452399,1.273237,-0.159217,0.501484,-0.630672,-1.734123,0.0,-0.700515,-0.700517,1.388213,0.599358,True
4,5,5,2015-07-31,4822,559,0.452399,1.273237,-0.159217,0.501484,-0.630672,-1.734123,0.0,-0.700515,-0.700517,1.388213,0.599358,True


## Train-test Split

In [12]:
# Train-test split
X_train, X_test, y_train, y_test = pd.split_data(data, feature_columns, 'Sales')

## Train Random Forest model

In [13]:
import model_training as mt
# Train Random Forest model
pipeline = mt.build_model_pipeline()
trained_pipeline = mt.train_model(pipeline, X_train, y_train)

## Evaluate model

In [14]:
import model_evaluation as ev
mae, y_pred = ev.evaluate_model(trained_pipeline, X_test, y_test)
print(f'Mean Absolute Error: {mae}')

Mean Absolute Error: 1595.1713566297485


## Feature importance

In [15]:
# Feature importance
importance_df = ev.feature_importance(trained_pipeline, feature_columns)
importance_df

,Feature,Importance
8,Open,0.770227
9,Promo,0.122902
2,DaysToHoliday,0.058462
0,Weekday,0.036357
4,BeginningOfMonth,0.003225
7,Season,0.002788
5,MidMonth,0.002509
6,EndOfMonth,0.002339
1,IsWeekend,0.000935
10,StateHoliday,0.000258
